bug scecret trong file note4_api: cách để cấu trúc: API vẫn xài trực tiếp các file scripts. Bug ko hiểu ở đâu

# 1. Break các scripts có sẵn thành cấu trúc api, services, config. ---- 


2. Cập nhật cấu trúc thư mục:

```bash
backend/
├── .env.example                    # File chứa các biến môi trường
├── app.py                   # File chính
├── api/                     # Thư mục chứa các API endpoints
│   ├── __init__.py
│   ├── ipa.py              # API cho IPA generation
│   ├── meaning.py          # API cho meaning exercises
│   ├── story.py            # API cho story generation
│   └── image.py            # API cho image generation
├── services/               # Business logic
│   ├── __init__.py
│   ├── openai_service.py   # OpenAI integration
│   ├── tts_service.py      # Text-to-speech service
│   └── image_service.py    # Image generation service
└── utils/                  # Utilities remain unchanged
```



# 2. Cách làm lươi hơn - Có làm đơn giản hơn bằng việc chỉ việc click vào nút trên UI thì nó chạy các file .py trong @scripts  không?? 


Dưới đây là bảng tóm tắt các cách xử lý đường dẫn trong project của bạn theo định dạng Markdown:

| **Phương pháp** | **Mô tả** | **Ví dụ mã nguồn** | **Ưu điểm** |
|------------------|-----------|---------------------|--------------|
| **Cách 1: Sử dụng `os.path.join`** | Dùng hàm `os.path.join` để xây dựng đường dẫn tương đối đến file `data.xlsx` trong thư mục `uploads`. | ```python data = pd.read_excel(os.path.join('..', 'uploads', 'data.xlsx'))``` | Đơn giản, dễ hiểu nhưng có thể gây lỗi nếu script chạy từ một thư mục khác. |
| **Cách 2: Đường dẫn tương đối trực tiếp** | Sử dụng đường dẫn tương đối để trỏ đến file `data.xlsx` từ các script khác nhau. | ```python data = pd.read_excel('../uploads/data.xlsx')``` | Nhanh chóng và dễ dùng, nhưng không ổn định nếu cấu trúc thư mục thay đổi. |
| **Cách 3: Sử dụng `Path` từ thư viện `pathlib`** | Tạo đường dẫn một cách an toàn và chính xác bằng cách sử dụng `Path`. | ```python from pathlib import Path UPLOAD_FOLDER = Path(__file__).parent.parent / 'uploads' data = pd.read_excel(UPLOAD_FOLDER / 'data.xlsx')``` | Đảm bảo tìm đúng đường dẫn bất kể script được chạy từ đâu; an toàn hơn và dễ bảo trì. |

**Ghi chú**: Cách thứ 3 được khuyến nghị sử dụng vì nó đảm bảo sự ổn định và khả năng bảo trì của dự án, đặc biệt trong các hệ thống phức tạp.

# 3. Về API của các phần xử lý file - tách các api này ra 1 file riêng refactor cho dễ nhỉ, như kiểu scripts ý 

Tôi sẽ giúp bạn tách các API liên quan đến file handling thành một blueprint riêng.

1. Đầu tiên, tạo file mới `backend/api/files/routes.py`:

```python:backend/api/files/routes.py
from flask import Blueprint, request, jsonify, send_file
import os
from werkzeug.utils import secure_filename
import pandas as pd

bp = Blueprint('files', __name__)

UPLOAD_FOLDER = './uploads'
OUTPUT_FOLDER = './output'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

@bp.route('/upload', methods=['POST'])
def upload_file():
    try:
        if 'file' not in request.files:
            return jsonify({'error': 'No file uploaded'}), 400

        file = request.files['file']
        if not file or file.filename == '':
            return jsonify({'error': 'No file selected'}), 400

        if not file.filename.endswith(('.xlsx', '.xls')):
            return jsonify({'error': 'Invalid file type. Please upload Excel file'}), 400

        filename = secure_filename(file.filename)
        file_path = os.path.join(UPLOAD_FOLDER, filename)
        file.save(file_path)
        
        df = pd.read_excel(file_path)
        data = df.to_dict(orient='records')
        
        return jsonify({
            'success': True,
            'data': data, 
            'filename': filename
        })
        
    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        }), 500

@bp.route('/update', methods=['POST'])
def update_file():
    data = request.json['data']
    filename = request.json['filename']

    df = pd.DataFrame(data)
    output_path = os.path.join(OUTPUT_FOLDER, filename)
    df.to_excel(output_path, index=False)
    
    return jsonify({'message': 'File updated successfully', 'filename': filename})

@bp.route('/list/<folder>', methods=['GET'])
def list_files(folder):
    try:
        if folder not in ['uploads', 'output']:
            return jsonify({'error': 'Invalid folder'}), 400
            
        folder_path = UPLOAD_FOLDER if folder == 'uploads' else OUTPUT_FOLDER
        files = []
        
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                file_stats = os.stat(file_path)
                files.append({
                    'name': filename,
                    'size': file_stats.st_size,
                    'modified': file_stats.st_mtime
                })
                
        return jsonify({
            'success': True,
            'files': files
        })
        
    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        }), 500

@bp.route('/download/<folder>/<filename>', methods=['GET'])
def download_folder_file(folder, filename):
    try:
        if folder not in ['uploads', 'output']:
            return jsonify({'error': 'Invalid folder'}), 400
            
        folder_path = UPLOAD_FOLDER if folder == 'uploads' else OUTPUT_FOLDER
        file_path = os.path.join(folder_path, secure_filename(filename))
        
        if not os.path.exists(file_path):
            return jsonify({'error': 'File not found'}), 404
            
        return send_file(file_path, as_attachment=True)
        
    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        }), 500
```

2. Tạo file `backend/api/files/__init__.py`:

```python:backend/api/files/__init__.py
from .routes import bp
```

3. Cập nhật file `backend/app.py`:

```python:backend/app.py
from flask import Flask
from flask_cors import CORS
from api.scripts import bp as scripts_bp
from api.files import bp as files_bp

app = Flask(__name__)
CORS(app)

# Register blueprints
app.register_blueprint(scripts_bp, url_prefix='/api/scripts')
app.register_blueprint(files_bp, url_prefix='/api/files')

if __name__ == '__main__':
    app.run(debug=True)
```

Giải thích các thay đổi:
1. Tất cả logic xử lý file được chuyển vào blueprint riêng
2. Các endpoints mới sẽ có prefix là `/api/files`
3. Các đường dẫn API mới sẽ là:
   - Upload: `POST /api/files/upload`
   - Update: `POST /api/files/update`
   - List files: `GET /api/files/list/<folder>`
   - Download: `GET /api/files/download/<folder>/<filename>`
4. Cấu trúc thư mục rõ ràng và dễ maintain hơn

Đảm bảo tạo đủ cấu trúc thư mục như sau:
```
backend/
├── api/
│   ├── files/
│   │   ├── __init__.py
│   │   └── routes.py
│   └── scripts/
│       ├── __init__.py
│       └── routes.py
└── app.py
```

Here are the `curl` commands for testing the API endpoints defined in your Flask application:

1. **Upload File:**

```bash
curl -X POST http://localhost:5000/api/files/upload \
  -F "file=@path/to/your/file.xlsx"
```

Replace `path/to/your/file.xlsx` with the actual path to the Excel file you want to upload.

2. **Update File:**

```bash
curl -X POST http://localhost:5000/api/files/update \
  -H "Content-Type: application/json" \
  -d '{
        "data": [{"column1": "value1", "column2": "value2"}],
        "filename": "your_file.xlsx"
      }'
```

Replace the JSON data with the actual data you want to update and `your_file.xlsx` with the filename.

3. **List Files in a Folder:**

```bash
curl http://localhost:5000/api/files/list/uploads
```

Replace `uploads` with `output` if you want to list files in the output folder.

4. **Download File:**

```bash
curl -O http://localhost:5000/api/files/download/uploads/your_file.xlsx
```

Replace `uploads` with `output` if the file is in the output folder, and `your_file.xlsx` with the actual filename you want to download.

# Thiết lập đường dẫn 

Dấu `../` trong đường dẫn là một ký hiệu được sử dụng trong hệ thống tệp để chỉ đến thư mục cha của thư mục hiện tại. Đây là cách để di chuyển lên một cấp trong cấu trúc thư mục.

### Ví dụ:

- Nếu bạn có cấu trúc thư mục như sau:

```
project/
├── backend/
│   ├── api/
│   │   └── files/
│   └── uploads/
└── frontend/
```

- Nếu bạn đang ở trong thư mục `files` và bạn muốn truy cập vào thư mục `uploads`, bạn có thể sử dụng đường dẫn `../../uploads`. 

### Giải thích:

- `..` đại diện cho thư mục cha của thư mục hiện tại.
- `../..` có nghĩa là đi lên hai cấp trong cấu trúc thư mục.

### Cách sử dụng trong mã:

Khi bạn sử dụng `os.path.join(os.path.dirname(__file__), '../../uploads')`, bạn đang tạo một đường dẫn đến thư mục `uploads` từ thư mục `files` bằng cách đi lên hai cấp. 

Điều này rất hữu ích khi bạn cần truy cập các tệp hoặc thư mục nằm ở các cấp khác nhau trong cấu trúc thư mục của dự án.

# CẤU TRÚC RESPONSE JSON KHI UPLOAD FILE THÀNH CÔNG


Dữ liệu phản hồi bạn cung cấp có cấu trúc đúng và có các thuộc tính cần thiết như `data`, `filename`, và `success`. Dưới đây là một số điểm quan trọng từ phản hồi này:

1. **Cấu trúc phản hồi:**
   - `data`: Là một mảng chứa các đối tượng, mỗi đối tượng có thông tin về câu hỏi chính, tình huống, từ vựng, và các thông tin khác.
   - `filename`: Tên của tệp mà dữ liệu được lưu trữ.
   - `success`: Trạng thái cho biết liệu yêu cầu đã thành công hay không.

2. **Thông tin trong `data`:**
   - Mỗi đối tượng trong mảng `data` chứa các thuộc tính như `MAIN QUESTION`, `situation`, `vocabulary`, và nhiều thuộc tính khác liên quan đến nội dung câu hỏi và ngữ cảnh.

3. **Trạng thái thành công:**
   - `success: true` cho biết rằng yêu cầu đã được thực hiện thành công và dữ liệu đã được trả về đúng cách.

### Ví dụ về cách xử lý phản hồi này trong mã:

Nếu bạn muốn xử lý phản hồi này trong mã JavaScript, bạn có thể làm như sau:

```javascript
const response = {
    data: [
        // ... (dữ liệu như bạn đã cung cấp)
    ],
    filename: "data_-_Copy.xlsx",
    success: true
};

if (response.success) {
    console.log("Dữ liệu đã được tải thành công:", response.data);
    console.log("Tên tệp:", response.filename);
} else {
    console.error("Có lỗi xảy ra khi tải dữ liệu.");
}
```

### Kết luận:
Phản hồi này có cấu trúc hợp lệ và có thể được sử dụng để xử lý dữ liệu trong ứng dụng của bạn. Nếu bạn gặp lỗi trong quá trình xử lý, hãy kiểm tra lại cách bạn truy cập các thuộc tính trong phản hồi để đảm bảo rằng chúng tồn tại và có giá trị như mong đợi.